# Prediction of the evolution of creatinine rates

## 0. Load useful modules

In [1]:
import numpy as np
from pandas import pandas as pd
#import time
#import copy
#from pprint import pprint

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, confusion_matrix, classification_report

#from sklearn.model_selection import cross_val_score, train_test_split, StratifiedShuffleSplit, GridSearchCV
#from sklearn.metrics import fbeta_score, make_scorer, classification_report, confusion_matrix
#from sklearn.ensemble import AdaBoostClassifier
#from xgboost import XGBClassifier

from termcolor import colored, cprint

In [2]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

## 1. Data preprocessing

### Load dataset

In [3]:
# Load dataset
data = pd.read_csv("dataset_with_labels.csv", engine='python').drop('Unnamed: 0',axis=1).reset_index(drop=True)
print('Initial number of examples : ', data.shape[0])
print('Initial number of columns : ', data.shape[1])
print()

# Remove columns that are ids
id_cols = [c for c in data.columns.values if ('_id' in c)]
print('Dropping ' + str(len(id_cols)) + ' columns that are not features : ')
print(id_cols)
for c in id_cols:
    data = data.drop(c,axis=1)
print('After dropping ids, number of columns : ', data.shape[1])
data.head()

Initial number of examples :  36251
Initial number of columns :  64

Dropping 2 columns that are not features : 
['subject_id', 'icustay_id']
After dropping ids, number of columns :  62


,creatinine,age,arterial_pressure_systolic,arterial_pressure_systolic_delay,arterial_pressure_diastolic,arterial_pressure_diastolic_delay,heart_rate,heart_rate_delay,weight_daily,weight_daily_delay,...,bilirubin,bilirubin_delay,c_reactive_protein,c_reactive_protein_delay,ethnicity,diagnosis,gender,creatinine_yesterday,creatinine_before_yesterday,label
0,0.6,58.363217,155.0,1800.0,75.0,1800.0,58.0,1800.0,93.7,73380.0,...,NaN,NaN,NaN,NaN,WHITE,INTRACRANIAL HEMORRHAGE,M,NaN,0.7,2.0
1,0.6,52.688716,165.0,1440.0,75.0,1440.0,102.0,1440.0,110.0,83700.0,...,NaN,NaN,NaN,NaN,WHITE,S/P BOATING ACCIDENT,M,0.6,0.6,2.0
2,2.0,67.864024,84.0,4200.0,50.0,4200.0,104.0,4260.0,115.8,158820.0,...,NaN,NaN,NaN,NaN,WHITE,SEPSIS,M,1.9,NaN,1.0
3,0.7,72.399244,120.0,60.0,74.0,60.0,103.0,60.0,NaN,NaN,...,NaN,NaN,NaN,NaN,WHITE,LOWER GASTROINTESTINAL BLEED,M,NaN,NaN,2.0
4,1.4,53.707219,144.0,900.0,75.0,900.0,67.0,900.0,54.6,102720.0,...,NaN,NaN,NaN,NaN,WHITE,ALCHOLIC CIRRHOSIS\EGD ** REMOTE EAST STONEMAN...,M,NaN,1.3,2.0


### Handle missing values

In [4]:
# Throw features that have a rate of missing values > x% 
max_missing_rate = 0.3
missing_rates = data.apply(lambda x : x.isnull(), axis=1).sum(axis=0)/data.shape[0]
to_drop = missing_rates[missing_rates>0.3].index.values

print('Following columns have missing rate >' + str(int(100*max_missing_rate)) + '% and will be dropped : ')
print(to_drop)
data = data.drop(labels=to_drop,axis=1)
print('Number of columns kept : ', data.shape[1])
print(data.columns.values)
print()

# Then drop lines where there remain some missing values
print('Drop lines where there are some NAs remaining...')
data = data.dropna(axis=0,how='any').reset_index(drop=True)
print('Number of lines kept : ', data.shape[0])
print()

# Check there is not missing value left in dataset
print('Number of missing values remaining : ', data.isnull().sum().sum())

Following columns have missing rate >30% and will be dropped : 
['weight_daily' 'weight_daily_delay' 'urine_output' 'urine_output_delay'
 'day_urine_output' 'day_urine_output_delay' 'scr' 'scr_delay' 'sodium'
 'sodium_delay' 'potassium' 'potassium_delay' 'calcium' 'calcium_delay'
 'phosphor' 'phosphor_delay' 'hemoglobine' 'hemoglobine_delay' 'uric_acid'
 'uric_acid_delay' 'chloride' 'chloride_delay' 'platelet_count'
 'platelet_count_delay' 'fibrinogen' 'fibrinogen_delay' 'urinary_sodium'
 'urinary_sodium_delay' 'urinary_potassium' 'urinary_potassium_delay'
 'urine_creatinin' 'urine_creatinin_delay' 'alkaline_phospatase'
 'alkaline_phospatase_delay' 'total_protein_blood'
 'total_protein_blood_delay' 'albumin' 'albumin_delay'
 'total_protein_urine' 'total_protein_urine_delay' 'bilirubin'
 'bilirubin_delay' 'c_reactive_protein' 'c_reactive_protein_delay'
 'creatinine_yesterday' 'creatinine_before_yesterday']
Number of columns kept :  16
['creatinine' 'age' 'arterial_pressure_systolic'
 'a

### Encode categorical variables

In [5]:
# Before performing dummy encoding, map some categorical variables to fewer modalities
# This is to control the number of features
print('Mapping modalities of ethnicity towards simplified modalities...')
simple_ethnicity = {
   'BLACK/AFRICAN AMERICAN': 'BLACK', 
   'WHITE': 'WHITE', 
   'UNKNOWN/NOT SPECIFIED': 'UNKNOWN',
   'HISPANIC/LATINO - DOMINICAN': 'OTHER', 
   'UNABLE TO OBTAIN': 'UNKNOWN',
   'PATIENT DECLINED TO ANSWER': 'UNKNOWN', 
   'ASIAN - CHINESE': 'ASIAN',
   'AMERICAN INDIAN/ALASKA NATIVE': 'OTHER', 
   'MULTI RACE ETHNICITY': 'OTHER',
   'WHITE - OTHER EUROPEAN': 'WHITE', 
   'OTHER': 'OTHER', 
   'PORTUGUESE': 'WHITE',
   'HISPANIC OR LATINO': 'OTHER', 
   'ASIAN': 'ASIAN', 
   'HISPANIC/LATINO - PUERTO RICAN': 'OTHER',
   'MIDDLE EASTERN': 'OTHER', 
   'ASIAN - KOREAN': 'ASIAN', 
   'BLACK/HAITIAN': 'BLACK',
   'ASIAN - OTHER': 'ASIAN', 
   'HISPANIC/LATINO - CUBAN': 'OTHER', 
   'ASIAN - FILIPINO': 'ASIAN',
   'BLACK/CAPE VERDEAN': 'BLACK', 
   'WHITE - BRAZILIAN': 'WHITE', 
   'ASIAN - ASIAN INDIAN': 'ASIAN',
   'WHITE - EASTERN EUROPEAN': 'WHITE', 
   'HISPANIC/LATINO - GUATEMALAN': 'OTHER',
   'ASIAN - VIETNAMESE': 'ASIAN', 
   'HISPANIC/LATINO - MEXICAN': 'OTHER',
   'WHITE - RUSSIAN': 'WHITE', 
   'BLACK/AFRICAN': 'BLACK', 
   'ASIAN - CAMBODIAN': 'ASIAN',
   'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE': 'OTHER'
}

data.loc[:,'ethnicity'] = data['ethnicity'].map(simple_ethnicity)
print('Remaining values for ethnicity : ')
print(data['ethnicity'].value_counts(dropna=False))
print()

print('Mapping modalities of diagnosis towards simplified modalities...')
to_sepsis = data['diagnosis'].apply(lambda x : 'FEVER' in x)
data.loc[to_sepsis,'diagnosis'] = 'SEPSIS'

to_resp_failure = data['diagnosis'].apply(lambda x : (('DYSPNEA' in x) | ('SHORTNESS OF BREATH' in x)))
data.loc[to_resp_failure,'diagnosis'] = 'RESPIRATORY FAILURE'

diag_to_keep = ['PNEUMONIA', 'CONGESTIVE HEART FAILURE', 'SUBARACHNOID HEMORRHAGE',
              'INTRACRANIAL HEMORRHAGE', 'ALTERED MENTAL STATUS', 'CORONARY ARTERY DISEASE',
              'ABDOMINAL PAIN', 'CHEST PAIN', 'HYPOTENSION', 'ACUTE RENAL FAILURE',
              'RESPIRATORY FAILURE', 'GASTROINTESTINAL BLEED', 'PANCREATITIS', 'SEPSIS']
to_other = data['diagnosis'].apply(lambda x : x not in diag_to_keep)
data.loc[to_other, 'diagnosis'] = 'OTHER'

print('Remaining values of diagnosis : ')
print(data['diagnosis'].value_counts(dropna=False))

Mapping modalities of ethnicity towards simplified modalities...
Remaining values for ethnicity : 
WHITE      20968
UNKNOWN     2565
BLACK       2111
OTHER       1787
ASIAN        654
NaN           42
Name: ethnicity, dtype: int64

Mapping modalities of diagnosis towards simplified modalities...
Remaining values of diagnosis : 
OTHER                       20856
SEPSIS                       1280
PNEUMONIA                    1169
INTRACRANIAL HEMORRHAGE       848
RESPIRATORY FAILURE           644
SUBARACHNOID HEMORRHAGE       637
ALTERED MENTAL STATUS         487
ABDOMINAL PAIN                402
PANCREATITIS                  383
CHEST PAIN                    328
CORONARY ARTERY DISEASE       295
CONGESTIVE HEART FAILURE      273
HYPOTENSION                   216
GASTROINTESTINAL BLEED        179
ACUTE RENAL FAILURE           130
Name: diagnosis, dtype: int64


In [6]:
# Perform dummy encoding
dummy_variables = ['ethnicity','diagnosis','gender']
print('Performing dummy encoding for features : ')
print(dummy_variables)
data = pd.get_dummies(data,columns=dummy_variables)
print('New list of columns :')
print(data.columns.values)

Performing dummy encoding for features : 
['ethnicity', 'diagnosis', 'gender']
New list of columns :
['creatinine' 'age' 'arterial_pressure_systolic'
 'arterial_pressure_systolic_delay' 'arterial_pressure_diastolic'
 'arterial_pressure_diastolic_delay' 'heart_rate' 'heart_rate_delay'
 'temperature' 'temperature_delay' 'ph_blood' 'ph_blood_delay' 'label'
 'ethnicity_ASIAN' 'ethnicity_BLACK' 'ethnicity_OTHER' 'ethnicity_UNKNOWN'
 'ethnicity_WHITE' 'diagnosis_ABDOMINAL PAIN'
 'diagnosis_ACUTE RENAL FAILURE' 'diagnosis_ALTERED MENTAL STATUS'
 'diagnosis_CHEST PAIN' 'diagnosis_CONGESTIVE HEART FAILURE'
 'diagnosis_CORONARY ARTERY DISEASE' 'diagnosis_GASTROINTESTINAL BLEED'
 'diagnosis_HYPOTENSION' 'diagnosis_INTRACRANIAL HEMORRHAGE'
 'diagnosis_OTHER' 'diagnosis_PANCREATITIS' 'diagnosis_PNEUMONIA'
 'diagnosis_RESPIRATORY FAILURE' 'diagnosis_SEPSIS'
 'diagnosis_SUBARACHNOID HEMORRHAGE' 'gender_F' 'gender_M']


### Remove outliers

In [7]:
feature_ranges = {
    'creatinine': {'min': 0.0, 'max': 20.0},
    'creatinine_yesterday': {'min': 0.0, 'max': 20.0},
    'creatinine_before_yesterday': {'min': 0.0, 'max': 20.0},
    'potassium': {'min': 1.2, 'max': 7.0},
    'ph_blood': {'min': 0.0, 'max': 14.0},
    'age': {'min': 0.0, 'max': 110.0},
    'bilirubin': {'min': 0.0, 'max': 20.0}
}

print('Initial number of examples : ', data.shape[0])
for k in feature_ranges:
    if k in data.columns.values:
        print('Dropping outliers for feature : ', k)
        data = data.loc[((data[k]>feature_ranges[k]['min']) & (data[k]<feature_ranges[k]['max'])),:]
        print('Remaining number of examples : ', data.shape[0])

Initial number of examples :  28127
Dropping outliers for feature :  creatinine
Remaining number of examples :  28122
Dropping outliers for feature :  ph_blood
Remaining number of examples :  28122
Dropping outliers for feature :  age
Remaining number of examples :  27313


### Normalize data

In [8]:
# Create a sklearn StandardScaler() for all numeric variables
numeric_variables = ['creatinine', 'age', 'arterial_pressure_systolic', 'arterial_pressure_diastolic',
                     'heart_rate', 'temperature', 'ph_blood']
print('Standardizing following features : ', numeric_variables)
numeric_scaler = StandardScaler()
data.loc[:,numeric_variables] = numeric_scaler.fit_transform(data.loc[:,numeric_variables])
for c,col in (data.loc[:,numeric_variables].describe(percentiles=[]).loc[['mean','std'],:]).iteritems():
    print('--- Feature : ', c)
    print(col)
print()

# Special case for delays : as they are all times in seconds, create one unique scaler for all delays
# (i.e. apply same transformation to each time measurement, such that they remain comparable)
time_variables = [i for i in data.columns.values if '_delay' in i]
print('Standardizing following features : ', time_variables)
time_scaler = StandardScaler()
time_scaler.fit(data.loc[:,time_variables].values.flatten())
data.loc[:,time_variables] = time_scaler.transform(data.loc[:,time_variables])

for c,col in (data.loc[:,time_variables].describe(percentiles=[]).loc[['mean','std'],:]).iteritems():
    print('--- Feature : ', c)
    print(col)
print()

Standardizing following features :  ['creatinine', 'age', 'arterial_pressure_systolic', 'arterial_pressure_diastolic', 'heart_rate', 'temperature', 'ph_blood']
--- Feature :  creatinine
mean    1.264320e-16
std     1.000018e+00
Name: creatinine, dtype: float64
--- Feature :  age
mean   -4.531781e-16
std     1.000018e+00
Name: age, dtype: float64
--- Feature :  arterial_pressure_systolic
mean    6.243556e-17
std     1.000018e+00
Name: arterial_pressure_systolic, dtype: float64
--- Feature :  arterial_pressure_diastolic
mean    1.248711e-17
std     1.000018e+00
Name: arterial_pressure_diastolic, dtype: float64
--- Feature :  heart_rate
mean   -3.080155e-16
std     1.000018e+00
Name: heart_rate, dtype: float64
--- Feature :  temperature
mean    1.601472e-15
std     1.000018e+00
Name: temperature, dtype: float64
--- Feature :  ph_blood
mean    1.673728e-15
std     1.000018e+00
Name: ph_blood, dtype: float64

Standardizing following features :  ['arterial_pressure_systolic_delay', 'arterial

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


### Prepare X and y input datasets
Split dataset into three sets :
- **hyperopt set (80%)** : for each model, perform grid search with cross-validation for hyperparameter optimization.
- **compare set (10%)** : compare performances obtained on this set for each model (with optimized hyperparameters)
- **test set (10%)** : in the end, check for overfitting by comparing the change in performances between compare_set and test_set

In [9]:
# Separate label from features
y = data['label']
X = data.drop('label',axis=1)

# Split into hyperopt, compare and test sets
X_hyperopt, X1, y_hyperopt, y1 = train_test_split(X, y, train_size=0.8, stratify=y.values)
X_compare, X_test, y_compare, y_test = train_test_split(X1, y1, train_size=0.5, stratify=y1.values)

print('--- Hyperopt set :')
print('Number of examples : ', X_hyperopt.shape[0])
print('Class balance : ')
print(y_hyperopt.value_counts(dropna=False))
print('--- Compare set :')
print('Number of examples : ', X_compare.shape[0])
print('Class balance : ')
print(y_compare.value_counts(dropna=False))
print('--- Test set :')
print('Number of examples : ', X_test.shape[0])
print('Class balance : ')
print(y_test.value_counts(dropna=False))


--- Hyperopt set :
Number of examples :  21850
Class balance : 
2.0    13337
1.0     4702
0.0     3811
Name: label, dtype: int64
--- Compare set :
Number of examples :  2731
Class balance : 
2.0    1667
1.0     588
0.0     476
Name: label, dtype: int64
--- Test set :
Number of examples :  2732
Class balance : 
2.0    1667
1.0     588
0.0     477
Name: label, dtype: int64


## 2. Hyperparameter tuning

### Initialize models and their respective set of hyperparameters for grid search
- Create list of dict : [{**'model_name'**: sklearn_model, **'hyperparam_grid'**:{**'hyperparam_name'**:[list of possible values]}}]
- then just loop on this list to perform hyperparameter optim for each model

In [47]:
# Classifiers and their respective grids for hyperparameter tuning should be specified here :
models = {
    'linear_SVM': {
        'classifier': OneVsRestClassifier(SGDClassifier(loss='hinge', penalty='l2', learning_rate='optimal', 
                                                        class_weight='balanced'), n_jobs=-1),
        'search_grid': {
            'estimator__alpha': [0.000001,0.000003,0.00001,0.00003,0.0001,0.0003,0.001, 0.003, 0.1, 0.3, 1.0]   
        }
    },
    'logistic_regression': {
        'classifier': OneVsRestClassifier(SGDClassifier(loss='log', penalty='l2', learning_rate='optimal', 
                                                        class_weight='balanced'), n_jobs=-1),
        'search_grid': {
            'estimator__alpha': [0.000001,0.000003,0.00001,0.00003,0.0001,0.0003,0.001, 0.003, 0.1, 0.3, 1.0]
        }
    },
    'logistic_regression_elasticnet': {
        'classifier': OneVsRestClassifier(SGDClassifier(loss='log', penalty='elasticnet', learning_rate='optimal',
                                                        class_weight='balanced'), n_jobs=-1),
        'search_grid': {
            'estimator__alpha': [0.000001,0.000003,0.00001,0.00005,0.0001,0.0005,0.001],
            'estimator__l1_ratio': [0.15,0.3,0.6,1.0]
        }
    },
    'random_forest': {
        'classifier': RandomForestClassifier(n_estimators=10, max_depth=None),
        'search_grid': {
            'n_estimators': [int(x) for x in np.linspace(200,2000,10)],
            'max_depth': np.append([int(x) for x in np.linspace(10,100,10)], None)
        }
    }
}

### Grid search for each model
/!\ this is time-consuming, pay attention to save the intermediate results of the grid search in a file
- in the end, create a list containing the best classifier obtained for each model : [sklearn_model(best hyperparams), kerasclassifier(best_hyperparams),...]

In [48]:
# Create custom performance metrics that will be used for hyperparameter optimization
def sensitivity_increase(y,y_pred):
    conf_mat = confusion_matrix(y, y_pred)
    return conf_mat[0,0]/(conf_mat[0,0] + conf_mat[0,1] + conf_mat[0,2])

def specificity_decrease(y,y_pred):
    conf_mat = confusion_matrix(y, y_pred)
    return (conf_mat[0,0] + conf_mat[0,2] + conf_mat[2,0] + conf_mat[2,2])/(conf_mat[0,0] + conf_mat[0,2] 
                                                                            + conf_mat[2,0] + conf_mat[2,2]
                                                                           + conf_mat[0,1] + conf_mat[2,1])

# The dictionnary below can be used in GridSearchCV() for multiple scoring
# /!\ This is doable only with sklearn 0.19 (current stable release: 0.18)
# For now we use only sensitivity_increase for hyperparameter optimization
scores = {
    'sensitivity_increase': sensitivity_increase, #make_scorer(sensitivity_increase),
    'specificity_decrease': specificity_decrease#make_scorer(specificity_decrease)
}

In [ ]:
# Loop over each model and perform grid search to tune its hyperparameters
for i in models.keys():
    print('---- Grid search for ' + str(i) + ' ---')
    clf = models[i]['classifier']
    params = models[i]['search_grid']
    
    #gridsearch = GridSearchCV(estimator=clf, param_grid=params, refit=True, verbose=1)
    gridsearch = GridSearchCV(estimator=clf, param_grid=params, scoring=make_scorer(sensitivity_increase), refit=True, verbose=1)
    gridsearch.fit(X_hyperopt,y_hyperopt)
    models[i]['best_classifier'] = gridsearch.best_estimator_
    models[i]['best_scores'] = gridsearch.best_score_
    models[i]['best_params'] = gridsearch.best_params_
    print('Best score : ', gridsearch.best_score_)
    print('Best set of hyperparameters : ', gridsearch.best_params_)


---- Grid search for linear_SVM ---
Fitting 3 folds for each of 11 candidates, totalling 33 fits


[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    9.4s finished


Best score :  0.32326838848777323
Best set of hyperparameters :  {'estimator__alpha': 3e-06}
---- Grid search for logistic_regression ---
Fitting 3 folds for each of 11 candidates, totalling 33 fits


[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:   11.0s finished


Best score :  0.3523860636469389
Best set of hyperparameters :  {'estimator__alpha': 3e-06}
---- Grid search for logistic_regression_elasticnet ---
Fitting 3 folds for each of 28 candidates, totalling 84 fits


[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:   28.6s finished


Best score :  0.3503124740986324
Best set of hyperparameters :  {'estimator__alpha': 1e-05, 'estimator__l1_ratio': 1.0}
---- Grid search for random_forest ---
Fitting 3 folds for each of 110 candidates, totalling 330 fits


## 3. Comparison of performances
Re-train models with their respective optimal sets of hyperparameters, on ALL the "hyperopt" dataset. Then assess and compare performances on the "compare" dataset

In [ ]:
compare_performances = pd.DataFrame(index=list(models.keys()),columns=list(scores.keys()))
for k in list(models.keys()):
    clf = models[k]['best_classifier']
    # Use ALL examples from "hyperopt" dataset to re-train model
    # NB: this is not mandatory as GridSearchCV(refit=True) does it.
    clf.fit(X_hyperopt,y_hyperopt)
    # Assess performances on "compare" dataset
    y_pred = clf.predict(X_compare)   
    for m in list(scores.keys()):
        compare_performances.loc[k,m] = scores[m](y_compare, y_pred)
print(compare_performances)

In [ ]:
print(models['random_forest'])

## 4.  Check for overfitting
Re-train models on ALL the examples from "hyperopt" and "compare", then assess performances on the "test" dataset.
NB : if there's a significant discrepancy in performances between "compare" and "test" datasets, we may be overfitting or suffering from bias. The generalized performances are the ones obtained on the "test" set.

In [ ]:
test_performances = pd.DataFrame(index=list(models.keys()),columns=list(scores.keys()))
for k in list(models.keys()):
    clf = models[k]['best_classifier']
    # Use ALL examples from "hyperopt" AND "compare" datasets to re-train models
    clf.fit(X_hyperopt.append(X_compare),y_hyperopt.append(y_compare))
    # Assess performances on "test" dataset
    y_pred = clf.predict(X_test)   
    for m in list(scores.keys()):
        test_performances.loc[k,m] = scores[m](y_test, y_pred)
        
print('Performances on "compare" set : ')
print(compare_performances)
print()
print('Performances on "test" set (= generalized) : ')
print(test_performances)

In [ ]:
print(X_test)

## 5. Alternative to be tested : Don't drop any missing values from the dataset but rather train an XGBoost classifier that handles missing values